In [ ]:
#@title install requirements for dreamlike-diffusion-1.0
!pip install diffusers==0.10.0
!pip install transformers scipy ftfy accelerate
!pip install "ipywidgets>=7,<8"

In [ ]:

#@title Dreamlike Diffusion
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
from numpy import int16

model_id = "dreamlike-art/dreamlike-diffusion-1.0"#@param {type:"string"}            
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")

g_cuda = None

g_cuda = torch.Generator(device='cuda')
seed = 12345 #@param {type:"number"}
g_cuda.manual_seed(seed)
prompt = "beautiful girl in cyberpunk corset, riot, cyberpyshco, {{{ultra realistic}}}, {{highly detailed}}, {photorealistic}, {blender}, massive scale, cyberpunk city, 8k, octane render, unreal engine, art by artgerm and yoshitaka amano" #@param {type:"string"}
negative_prompt = " (((duplicate))), ((morbid)), ((mutilated)), out of frame, extra fingers, mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), blurry, (((deformed))), ((ugly)), ((bad anatomy)), (((bad proportions))), ((extra limbs)), cloned face, (((disfigured))), out of frame, ugly, extra limbs, (bad anatomy), gross proportions, (malformed limbs), ((missing arms)), ((missing legs)), (((extra arms))), (((extra legs))), mutated hands, (fused fingers), (too many fingers), (((long neck)))" #@param {type:"string"}
num_samples = 100 #@param {type:"number"}
guidance_scale = 10 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 768 #@param {type:"number"}

for i in range(num_samples):
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images
  seed += 100

  for img in images:
      img.save(f"Seed_{seed}.png")  
      display(img)

In [ ]:
#@title install requirements for Stable-diffusion-2.1
!pip install --upgrade git+https://github.com/huggingface/diffusers.git
# !pip install diffusers
!pip install --upgrade git+https://github.com/huggingface/transformers/
# !pip install transformers
!pip install accelerate==0.12.0
!pip install scipy
!pip install ftfy
!pip install gradio -q

import os
from subprocess import getoutput

os.system("pip install --extra-index-url https://download.pytorch.org/whl/cu113 torch torchvision==0.13.1+cu113")
os.system("pip install triton==2.0.0.dev20220701")
gpu_info = getoutput('nvidia-smi')
if ("A10G" in gpu_info):
    os.system(
        f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl")
elif ("T4" in gpu_info):
    os.system(
        f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl")

# ### install xformers
from IPython.utils import capture
from subprocess import getoutput
from re import search

with capture.capture_output() as cap:

     smi_out = getoutput('nvidia-smi')
     supported = search('(T4|P100|V100|A100|K80)', smi_out)

     if not supported:
       while True:
         print("\x1b[1;31mThe current GPU is not supported, try starting a new session.\x1b[0m")
     else:
       supported = supported.group(0)

!pip install -qq https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
!pip install -qq -U --pre triton

In [ ]:
#@title Stable-diffusion-2-1
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionUpscalePipeline, \
    DiffusionPipeline, StableDiffusionDepth2ImgPipeline, DPMSolverMultistepScheduler
import gradio as gr
import torch
from PIL import Image
import random
from torch import autocast
from IPython.display import display


# model_id = 'stabilityai/stable-diffusion-2'
model_id = 'stabilityai/stable-diffusion-2-1'

scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    revision="fp16",
    torch_dtype=torch.float16,
    scheduler=scheduler
).to("cuda")

pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()

g_cuda = None
g_cuda = torch.Generator(device="cuda")
seed = 12345 #@param{type:"number"}
g_cuda.manual_seed(seed)
num_samples = 10 #@param{type:"number"}
prompt = "beautiful girl in cyberpunk corset, riot, cyberpyshco, {{{ultra realistic}}}, {{highly detailed}}, {photorealistic}, {blender}, massive scale, cyberpunk city, 8k, octane render, unreal engine, art by artgerm and yoshitaka amano"#@param{type:"string"}
neg_prompt = " (((duplicate))), ((morbid)), ((mutilated)), out of frame, extra fingers, mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), blurry, (((deformed))), ((ugly)), ((bad anatomy)), (((bad proportions))), ((extra limbs)), cloned face, (((disfigured))), out of frame, ugly, extra limbs, (bad anatomy), gross proportions, (malformed limbs), ((missing arms)), ((missing legs)), (((extra arms))), (((extra legs))), mutated hands, (fused fingers), (too many fingers), (((long neck)))"#@param{type:"string"}
steps = 50 #@param{type:"number"}
guidance_scale = 10 #@param{type:"number"}
width = 768 #@param{type:"number"}
height = 768 #@param{type:"number"}

for i in range(num_samples):
    with autocast("cuda"):
        images = pipe(
            prompt = prompt,
            negative_prompt = neg_prompt,
            num_inference_steps = steps,
            guidance_scale = guidance_scale,
            width = width,
            height = height,
            generator = g_cuda
        ).images

    seed += 100

    for img in images:
        img.save(f"seed_{seed}.png")
        display(img)